# GEOQUERIES

In [1]:
from pymongo import MongoClient
from pymongo import GEOSPHERE
import json
import pandas as pd
from dotenv import load_dotenv
import os
import requests
import geopandas as gpd
from bson.json_util import dumps

In [2]:
#london_loc = {'type': 'Point', 'coordinates':[ -0.0870022262169723, 51.520624794291884]}
#singapore_loc = {'type': 'Point', 'coordinates':[103.84868192263733, 1.2862863534171092]}
#san_francisco_loc = {'type': 'Point', 'coordinates':[ -122.41049370727525,37.79787988216447]}
london_loc1 = [-0.0870022262169723, 51.520624794291884]
singapore_loc1 = [103.84868192263733, 1.2862863534171092]
san_francisco_loc1 =[ -122.41049370727525,37.79787988216447]

In [3]:
metros = 2500

In [4]:
#Crear la conexión con el database de Londres dentro de MongoDB
client = MongoClient("localhost:27017")
db = client.get_database("Ciudades")

In [5]:
ln = db.get_collection("London")
sg = db.get_collection("Singapore")
sf = db.get_collection("San Francisco")


# FUNCIONES

In [6]:
'''
Esta función recibe la coleccion con la que quieres trabajar, la localizacion establecida, el maximo de distancia
que quieres y la classificación que quieres buscar.
Te devuelve sitios cercanos a la ubicación ajustandose a las condiciones(maxima distancia, por classificacion y por
localización)

'''
def coger_sitios_cercanos(collection, location, Classification, max_distance):
    var_schools = {"$and":[{"location2": {"$near": {"$geometry": location,"$minDistance":0,"$maxDistance": max_distance}}},{"Classification": f'{Classification}'}]}
    encontrar_colegios = collection.find(var_schools)
    return list(encontrar_colegios)
    

In [7]:
#Tipos de classificación/Type
#1. School: Type
#2. Technological Startups
#3. Designing Studios
#4. Leisure Places
#5. Travelling Stations
#6. Parks

In [12]:
def get_distances(collection, location1):
    query = [{'$geoNear':{'near':location1,
    'distanceField' : "distance",
    'maxDistance': 0.1,
    'distanceMultiplier': 6371,
    'spherical': True}}]
    geoloc = collection.aggregate(query)
    response_json = json.loads(dumps(geoloc))
    df = pd.DataFrame(response_json)
    df1 = df[['Name', 'Classification', 'Type', 'Country', 'Adress', 'Latitude', 'Longitude', 'location2', 'distance']]
    return df1
    


In [26]:
a

,Name,Classification,Type,Country,Adress,Latitude,Longitude,location2,distance
0,Finsbury Square,Parks,Park,GB,Finsbury Sq (Finsbury Pavement),51.520694,-0.086512,"{'type': 'Point', 'coordinates': [-0.086511969...",0.034776
1,Finch's,Leisure Places,Pub,GB,12A Finsbury Sq (City Rd),51.521709,-0.087048,"{'type': 'Point', 'coordinates': [-0.087048251...",0.120590
2,The Flying Horse,Leisure Places,Pub,GB,52 Wilson St (Sun St),51.520375,-0.085176,"{'type': 'Point', 'coordinates': [-0.085176182...",0.129349
3,Singer Tavern,Leisure Places,Bar,GB,1 City Rd,51.522259,-0.086933,"{'type': 'Point', 'coordinates': [-0.086932681...",0.181763
4,Red Lion,Leisure Places,Pub,GB,1 Eldon St (South Pl),51.518843,-0.086181,"{'type': 'Point', 'coordinates': [-0.086181312...",0.206137
...,...,...,...,...,...,...,...,...,...
350,Bright Horizons Mount Carmel Day Nursery and P...,School,Nursery School,GB,18 Eden Grove,51.551394,-0.112786,"{'type': 'Point', 'coordinates': [-0.112786369...",3.858264
351,Monkey Puzzle Day Nursery,School,Nursery School,GB,"Loxfords House, Highbury Park, N5",51.556389,-0.098601,"{'type': 'Point', 'coordinates': [-0.098600901...",4.056888
352,City Academy Hackney,School,High School,GB,Homerton Row,51.548704,-0.048845,"{'type': 'Point', 'coordinates': [-0.048845102...",4.088336
353,Camden School for Girls,School,High School,GB,Sandall Street,51.547697,-0.134395,"{'type': 'Point', 'coordinates': [-0.134395257...",4.450611


In [14]:
a = get_distances(ln, london_loc1)

# PONDERACION

In [15]:
# Para escoger la localización, debemos dar importancia a las condiciones escogidas

In [24]:
parks = 0.05
school = 0.30
Technological_Startups = 0.05
Designing_Studios = 0.25
Travelling_Stations = 0.2
Leisure = 0.15